In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
import glob

import torch
print(torch.__version__)
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from functools import partial

import librosa
from datetime import datetime
from datetime import timedelta
import shutil

2.2.1+cu121


In [ ]:
pwd

'/content'

# Loading in Labeled Excel Files

In [ ]:
#Change path to where the coded files are stored in you Drive

path = "/your_path/ReCodedFiles/"

xlfiles = glob.glob(path + "*.xlsx")

In [ ]:
xlfiles

['/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/Copy of 522 TI Part 4.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/Copy of 554 T1 Part 2.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/Copy 531 T2 Part 2.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/209 T2 Part 3 updated.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/110 T2 Part 2 updated.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/138 T1 Part 1 updated.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/199 T1 updated.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/262 T2 Part 2 copy.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/287 T2 Part 2.xlsx',
 '/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/301 T1 Part 2  - Copy.xlsx']

# Data Manipulation Functions

In [ ]:
def convert_to_timedelta(time_str):
    # Split the string on the colon
    parts = time_str.split(':')

    # Parse minutes and seconds from the parts
    minutes = int(parts[0])
    seconds = int(parts[1]) if len(parts) > 1 else 0

    # Create a timedelta object
    time_delta = timedelta(minutes=minutes, seconds=seconds)

    return time_delta

In [ ]:
def save_fig(file, file_name, save_path):
    frame_size = 2048
    hopsize = 512
    y_axis = "linear"

    y = np.abs(file) ** 2
    y_log = librosa.power_to_db(y)
    plt.figure(figsize = (25,9))
    plt.axis('off')
    _ = librosa.display.specshow(y_log, sr = sample_rate, hop_length = hopsize,
                            x_axis = "time", y_axis = y_axis, )
    plt.savefig(save_path + file_name + ".jpg", bbox_inches = "tight", pad_inches = 0)
    plt.close()

In [ ]:
def gen_excel_data(file):
  print(str((file.split("/")[-1].split(".mp3")[0])))
  #Read in file
  df = pd.read_excel(file, usecols=[0, 1, 2], engine='openpyxl')
  # Drop rows where any cell is NA
  df_clean = df.dropna()
  data_as_list = df_clean.values.tolist()

  # List to store the new intervals
  new_intervals = []

  # Generate three-second intervals
  for row in data_as_list:
    if type(row[0]) == datetime.time:
      t1 = row[0]
      current_time = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
    else:
      current_time = convert_to_timedelta(row[0])
    if type(row[1]) == datetime.time:
      t2 = row[1]
      end_time = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
    else:
      end_time = convert_to_timedelta(row[1])
    classification = row[2]
    while current_time < end_time:
        # Calculate the next time point, without exceeding the end time
        next_time = min(current_time + pd.Timedelta(seconds=3), end_time)
        # Append the new interval to the list
        new_intervals.append([current_time, next_time, classification])
        current_time = next_time

  return new_intervals

In [ ]:
def process_data(file, custom_intervals, save_path_img):
    file_name = file.split("/")[-1].split(".mp3")[0]
    print("Processing: "+ file_name)
    # Load the entire audio file
    audio, sr = librosa.load(file, sr = 16000)
    frame_size = 2048
    hopsize = 512
    y_axis = "linear"

    # Process each custom interval
    for i, interval in enumerate(custom_intervals):
        # Extract timedelta objects and convert to total seconds
        start_sec, end_sec = [t.total_seconds() for t in interval[:2]]

        # Convert start and end times in seconds to samples
        start_sample = int(start_sec * sr)
        end_sample = int(end_sec * sr)

        # Ensure the interval does not exceed the audio length
        if end_sample > len(audio):
            print(f"End sample for interval {i} exceeds audio length. Skipping this interval.")
            continue

        # Extract the interval
        audio_interval = audio[start_sample:end_sample]

        # Perform the STFT on the current interval
        D = librosa.stft(audio_interval)
        D_magnitude = np.abs(D)
        D_db = librosa.amplitude_to_db(D_magnitude, ref=np.max)

        # Save as Image file
        plt.figure(figsize = (25,9))
        plt.axis('off')
        _ = librosa.display.specshow(D_db, sr = sr, hop_length = hopsize,
                            x_axis = "time", y_axis = y_axis, )
        plt.savefig(save_path_img + file_name + "(" + str(start_sec) + " to " + str(end_sec) + ") " + interval[2] + ".jpg", bbox_inches = "tight", pad_inches = 0)
        plt.close()

        # # Save the interval to a CSV file
        # interval_filename = f'{file}_interval_{i}.csv'
        # np.savetxt(interval_filename, D_db, delimiter=',')

        # Optional: Store the interval in a dictionary with a label if needed
        # train_mat_dict[interval_filename] = [D_db, "parent"]

        print(f"Processed interval {i} ({start_sec}s to {end_sec}s) of file {file}, {interval[2]}")

# Loading in .mp3 Files

In [ ]:
# Path to the directory containing CSV files
directory_path = "/your_mp3_file_path/"

# Get a list of all CSV files in the directory
mp3_files = glob.glob(directory_path + "*.mp3")

In [ ]:
len(mp3_files)

157

In [ ]:
test_files = [directory_path + '138 T1 part 1 child audio.mp3',
              directory_path + '199 T1 child audio.mp3',
              directory_path + '209 T2 part 3 child audio.mp3',
              directory_path + '262 T2 part 2 child audio.mp3',
              directory_path + '287 T2 part 2 child audio.mp3',
              directory_path + '301 T1 part 2 child audio.mp3',
              directory_path + '522 T1 part 4 child audio.mp3',
              directory_path + '554 T1 part 2 child audio.mp3',
              directory_path + '110 T2 part 2 child audio.mp3',
              directory_path + '531 T2 part 2 child audio.mp3']


138 T1 part 1 child audio Output

In [ ]:
filename = test_files[0]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[5])

138 T1 part 1 child audio
138 T1 Part 1 updated.xlsx


In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test1/"

process_data(filename, excel_file, spath)

Processing: 138 T1 part 1 child audio
Processed interval 0 (0.0s to 1.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, b
Processed interval 1 (1.0s to 4.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, i
Processed interval 2 (4.0s to 5.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, i
Processed interval 3 (5.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, b
Processed interval 4 (6.0s to 7.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, a
Processed interval 5 (7.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, b
Processed interval 6 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/138 T1 part 1 child audio.mp3, a
Processed interval 7 (12.0s to 14.0s)

199 T1 child audio Output

In [ ]:
filename2 = test_files[1]
print(filename2.split("/")[-1].split(".mp3")[0])
excel_file2 = gen_excel_data(xlfiles[6])

199 T1 child audio
199 T1 updated.xlsx


In [ ]:
#Set output path

spath2 = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test2/"

process_data(filename2, excel_file2, spath2)

Processing: 199 T1 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 1 (3.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 2 (6.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 3 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 4 (12.0s to 15.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 5 (15.0s to 18.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 6 (18.0s to 21.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/199 T1 child audio.mp3, b
Processed interval 7 (21.0s to 24.0s) of file /content/drive/My Drive/CapstoneNewExcelF

110 T2 part 2 child audio Output

In [ ]:
filename = test_files[8]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[4])

110 T2 part 2 child audio
110 T2 Part 2 updated.xlsx


In [ ]:
len(excel_file)

1322

In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test3/"

process_data(filename, excel_file, spath)

Processing: 110 T2 part 2 child audio
Processed interval 0 (0.0s to 2.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, b
Processed interval 1 (2.0s to 5.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, a
Processed interval 2 (5.0s to 8.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, a
Processed interval 3 (8.0s to 11.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, a
Processed interval 4 (11.0s to 14.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, a
Processed interval 5 (14.0s to 17.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, a
Processed interval 6 (17.0s to 20.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/110 T2 part 2 child audio.mp3, a
Processed interval 7 (20.0s to 

531 T2 part 2 child audio Output

In [ ]:
filename = test_files[9]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[2])

531 T2 part 2 child audio
Copy 531 T2 Part 2.xlsx


In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test4/"

process_data(filename, excel_file, spath)

Processing: 531 T2 part 2 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 1 (3.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 2 (6.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 3 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 4 (12.0s to 15.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 5 (15.0s to 18.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 6 (18.0s to 21.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/531 T2 part 2 child audio.mp3, b
Processed interval 7 (21.0s to 


554 T1 part 2 child audio Output

In [ ]:
filename = test_files[7]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[1])

554 T1 part 2 child audio
Copy of 554 T1 Part 2.xlsx


In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test4/"

process_data(filename, excel_file, spath)

Processing: 554 T1 part 2 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, b
Processed interval 1 (3.0s to 4.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, b
Processed interval 2 (4.0s to 7.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, a
Processed interval 3 (7.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, a
Processed interval 4 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, b
Processed interval 5 (12.0s to 15.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, b
Processed interval 6 (15.0s to 18.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/554 T1 part 2 child audio.mp3, b
Processed interval 7 (18.0s to 21

522 T1 part 4 child audio Output

In [ ]:
filename = test_files[6]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[0])

522 T1 part 4 child audio
Copy of 522 TI Part 4.xlsx


In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test5/"

process_data(filename, excel_file, spath)

Processing: 522 T1 part 4 child audio
Processed interval 0 (0.0s to 1.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, b
Processed interval 1 (1.0s to 4.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, i
Processed interval 2 (4.0s to 5.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, i
Processed interval 3 (5.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, b
Processed interval 4 (6.0s to 7.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, a
Processed interval 5 (7.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, b
Processed interval 6 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/522 T1 part 4 child audio.mp3, a
Processed interval 7 (12.0s to 14.0s)

262 T2 part 2 child audio Output

In [ ]:
filename = test_files[3]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[9])

262 T2 part 2 child audio
262 T2 Part 2 copy.xlsx


In [ ]:
len(excel_file)

1313

In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test6/"

process_data(filename, excel_file, spath)

Processing: 262 T2 part 2 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, i
Processed interval 1 (3.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, i
Processed interval 2 (6.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, i
Processed interval 3 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, i
Processed interval 4 (12.0s to 15.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, i
Processed interval 5 (15.0s to 16.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, i
Processed interval 6 (16.0s to 17.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/262 T2 part 2 child audio.mp3, a
Processed interval 7 (17.0s to 

Issues with 209, 287 , 301

In [ ]:
xlfiles[7]

'/content/drive/My Drive/CapstoneNewExcelFiles/ReCodedFiles/287 T2 Part 2.xlsx'

287 T2 part 2 child audio Output

In [ ]:
filename = test_files[4]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data_dt(xlfiles[7])

287 T2 part 2 child audio
287 T2 Part 2.xlsx


In [ ]:
len(excel_file)

2286

In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test7/"

process_data(filename, excel_file, spath)

Processing: 287 T2 part 2 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, i
Processed interval 1 (3.0s to 4.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, i
Processed interval 2 (4.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, a
Processed interval 3 (6.0s to 8.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, i
Processed interval 4 (8.0s to 10.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, a
Processed interval 5 (10.0s to 13.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, b
Processed interval 6 (13.0s to 16.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/287 T2 part 2 child audio.mp3, b
Processed interval 7 (16.0s to 17

301 T1 part 2 child audio Output

In [ ]:
filename = test_files[5]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data(xlfiles[9])

301 T1 part 2 child audio
301 T1 Part 2  - Copy.xlsx


In [ ]:
len(excel_file)

1551

In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test8/"

process_data(filename, excel_file, spath)

Processing: 301 T1 part 2 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 1 (3.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 2 (6.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 3 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 4 (12.0s to 15.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 5 (15.0s to 18.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 6 (18.0s to 21.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/301 T1 part 2 child audio.mp3, a
Processed interval 7 (21.0s to 

209 T2 part 3 child audio Output

Had to modify function for format of this excel file

In [ ]:
def gen_excel_data_dt(file):
  print(str((file.split("/")[-1].split(".mp3")[0])))
  #Read in file
  df = pd.read_excel(file, usecols=[0, 1, 2], engine='openpyxl')
  # Drop rows where any cell is NA
  df_clean = df.dropna()
  data_as_list = df_clean.values.tolist()

  format_string = "%H:%M:%S"

  # List to store the new intervals
  new_intervals = []

  # Generate three-second intervals
  for row in data_as_list:
    if type(row[0]) == datetime.time:
      t1 = row[0]
      current_time = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
    else:
      dt0 = datetime.strptime(row[0], format_string).time()
      current_time = timedelta(hours=dt0.hour, minutes=dt0.minute, seconds=dt0.second)
    if type(row[1]) == datetime.time:
      t2 = row[1]
      end_time = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
    else:
      dt1 = datetime.strptime(row[1], format_string).time()
      end_time = timedelta(hours=dt1.hour, minutes=dt1.minute, seconds=dt1.second)
    classification = row[2]
    while current_time < end_time:
        # Calculate the next time point, without exceeding the end time
        next_time = min(current_time + pd.Timedelta(seconds=3), end_time)
        # Append the new interval to the list
        new_intervals.append([current_time, next_time, classification])
        current_time = next_time

  return new_intervals

In [ ]:
filename = test_files[2]
print(filename.split("/")[-1].split(".mp3")[0])
excel_file = gen_excel_data_dt(xlfiles[3])

209 T2 part 3 child audio
209 T2 Part 3 updated.xlsx


In [ ]:
#Set output path

spath = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test9/"

process_data(filename, excel_file, spath)

Processing: 209 T2 part 3 child audio
Processed interval 0 (0.0s to 3.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 1 (3.0s to 6.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 2 (6.0s to 9.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 3 (9.0s to 12.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 4 (12.0s to 15.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 5 (15.0s to 18.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 6 (18.0s to 21.0s) of file /content/drive/My Drive/CapstoneNewExcelFiles/MP3Files/209 T2 part 3 child audio.mp3, b
Processed interval 7 (21.0s to 

# Organizing Image Files

In [ ]:
#Change path to where the coded files are stored in you Drive

path = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test1/"

image_files = glob.glob(path + "*.jpg")

In [ ]:
len(image_files)

1467

In [ ]:
for i in range(2, 10):
  path = f"/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test{i}/"
  new_files = glob.glob(path + "*.jpg")
  image_files.extend(new_files)
  print(i, len(new_files))

2 1392
3 1322
4 2771
5 1467
6 1313
7 2083
8 1541
9 1436


In [ ]:
len(image_files)

14792

In [ ]:
import os

# Specify the directory you want to search
directory = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test1/"

# List all entries in the directory
entries = os.listdir(directory)

# Count files (ignoring subdirectories)
file_count = sum(os.path.isfile(os.path.join(directory, entry)) for entry in entries)

print(f'There are {file_count} files in the directory.')

There are 1467 files in the directory.


# Organize Files to the Same Folder

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/FullData/"

for file in image_files:
  shutil.copy(file, destination_folder)

In [ ]:
# for file in image_files:
#   !mv "/content/drive/My Drive/your_source_folder/your_file.txt" "/content/drive/My Drive/your_destination_folder/your_file.txt"


'/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/Test1/138 T1 part 1 child audio(1118.0 to 1119.0) b.jpg'

In [ ]:
# Specify the directory you want to search
directory = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/FullData/"

# List all entries in the directory
entries = os.listdir(directory)

# Count files (ignoring subdirectories)
file_count = sum(os.path.isfile(os.path.join(directory, entry)) for entry in entries)

print(f'There are {file_count} files in the directory.')

There are 14792 files in the directory.


In [ ]:
#Change path to where the coded files are stored in you Drive

path = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/FullData/"

imagefiles = glob.glob(path + "*.jpg")

In [ ]:
len(imagefiles)

14792

In [ ]:
from sklearn.model_selection import train_test_split

Train splits

In [ ]:
i_files = []
a_files = []
b_files = []
o_files = []

for file in imagefiles:
  classification = file.split(" ")[-1].split(".")[0]
  if classification == 'i':
    i_files.append(file)
  elif classification == 'a':
    a_files.append(file)
  elif classification == 'b':
    b_files.append(file)
  else:
    o_files.append(file)

In [ ]:
len(i_files)

1261

In [ ]:
len(a_files)

3555

In [ ]:
len(o_files)

190

In [ ]:
len(b_files)

9786

Creating Large Dataset

In [ ]:
image_trainL_a, image_testL_a = train_test_split(a_files, test_size=0.2, random_state=49)
image_trainL_i, image_testL_i = train_test_split(i_files, test_size=0.2, random_state=49)
image_trainL_b, image_testL_b = train_test_split(b_files, test_size=0.2, random_state=49)
image_trainL_o, image_testL_o = train_test_split(o_files, test_size=0.2, random_state=49)

In [ ]:
print("number of infant image train data points: ", len(image_trainL_i))
print("number of infant image test data points: ", len(image_testL_i))
print("number of adult image train data points: ", len(image_trainL_a))
print("number of background image test data points: ", len(image_testL_a))
print("number of background image train data points: ", len(image_trainL_b))
print("number of background image test data points: ", len(image_testL_b))
print("number of other image train data points: ", len(image_trainL_o))
print("number of other image test data points: ", len(image_testL_o))

number of infant image train data points:  1008
number of infant image test data points:  253
number of adult image train data points:  2844
number of background image test data points:  711
number of background image train data points:  7828
number of background image test data points:  1958
number of other image train data points:  152
number of other image test data points:  38


# Export Image Files Back to Drive

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainLarge/i_train/"

for file in image_trainL_i:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestLarge/i_test/"

for file in image_testL_i:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainLarge/a_train/"

for file in image_trainL_a:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestLarge/a_test/"

for file in image_testL_a:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainLarge/b_train/"

for file in image_trainL_b:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestLarge/b_test/"

for file in image_testL_b:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainLarge/o_train/"

for file in image_trainL_o:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestLarge/o_test/"

for file in image_testL_o:
  shutil.copy(file, destination_folder)

# Creating Small Dataset

In [ ]:
import random

In [ ]:
a_files_samples = random.sample(a_files, len(i_files))

In [ ]:
len(a_files_samples)

1261

In [ ]:
b_files_samples = random.sample(b_files, len(i_files))

In [ ]:
len(b_files_samples)

1261

In [ ]:
b_files_samples

NameError: name 'b_files_samples' is not defined

Train and Test Split

In [ ]:
image_train_a, image_test_a = train_test_split(a_files_samples, test_size=0.2, random_state=49)
image_train_i, image_test_i = train_test_split(i_files, test_size=0.2, random_state=49)
image_train_b, image_test_b = train_test_split(b_files_samples, test_size=0.2, random_state=49)

In [ ]:
print("number of infant image train data points: ", len(image_train_i))
print("number of infant image test data points: ", len(image_test_i))
print("number of adult image train data points: ", len(image_train_a))
print("number of background image test data points: ", len(image_test_a))
print("number of background image train data points: ", len(image_train_b))
print("number of background image test data points: ", len(image_test_b))

number of infant image train data points:  1008
number of infant image test data points:  253
number of adult image train data points:  1008
number of background image test data points:  253
number of background image train data points:  1008
number of background image test data points:  253


# Export Small Dataset Back to Drive

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainSmall/i_train/"

for file in image_train_i:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestSmall/i_test/"

for file in image_test_i:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainSmall/a_train/"

for file in image_train_a:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestSmall/a_test/"

for file in image_test_a:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TrainSmall/b_train/"

for file in image_train_b:
  shutil.copy(file, destination_folder)

In [ ]:
destination_folder = "/content/drive/My Drive/CapstoneNewExcelFiles/ImageFiles/TestSmall/b_test/"

for file in image_test_b:
  shutil.copy(file, destination_folder)